In [1]:
#importing all libraries
import pandas as pd
import selenium
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [28]:
# loading Chrome webdriver
driver = webdriver.Chrome('chromedriver.exe')

* Q1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [3]:
# opening the url.

driver.get('https://www.amazon.in')

search_inp = input("Enter the product: ")
search_bar = driver.find_element_by_xpath("//input[@id='twotabsearchtextbox']") # locating search bar
search_bar.send_keys(search_inp)

# locating search button
search_btn = driver.find_element_by_xpath("//input[@id='nav-search-submit-button']")
search_btn.click()

Enter the product: sunglasses


* Q2. Now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical.

* Details to be scraped are:-
1. Brand Name.
2. Name of the product.
3. Rating.
4. Number of ratings.
5. Price.
6. Return/Exchange.
7. Expected Delivery.
8. Availability.
9. Other details.
10. Product url.

In [5]:
brand_name = []
product_name = []
rating = []
num_rating = []
ret_exc = []
exp_del = []
avai = []
price = []
other_dt = []

In [9]:
product_url = []

# first we scrap all the product url from page 1.
for a in driver.find_elements_by_xpath("//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-2']/a"):
        product_url.append(a.get_attribute('href'))
        
name = driver.find_element_by_xpath("//li[@class='a-last']/a") #storing the value of next page in name variable

for i in range(2):
    driver.get(name.get_attribute('href')) #passing the value of next page to the driver to connect to the next page.
    time.sleep(3)
    
    for j in driver.find_elements_by_xpath("//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-2']/a"): #storing next page product url
        product_url.append(j.get_attribute('href'))
    
    name = driver.find_element_by_xpath("//li[@class='a-last']/a") #now we again taking the url of next page.

In [11]:
from selenium.common.exceptions import NoSuchElementException
for i in product_url:
    driver.get(i)
    time.sleep(3)
    
    # sraping the product name
    try:
        pname = driver.find_element_by_xpath("//span[@class='a-size-large product-title-word-break']")
        product_name.append(pname.text)
    except NoSuchElementException:
        product_name.append('-')
        
    # scraping the rating of the product
    try:
        rat = driver.find_element_by_xpath("//span[@class='a-icon-alt']")
        rating.append(rat.text)
    except NoSuchElementException:
        rating.append('-')
    
    # scraping the number of ratings of the product    
    try:
        nrat = driver.find_element_by_xpath("//span[@class='a-size-base a-nowrap']//span")
        num_rating.append(nrat.text)
    except NoSuchElementException:
        num_rating.append('-')
        
    # scraping the avilability of the product    
    try:
        pr = driver.find_element_by_xpath("//span[@class='a-size-medium a-color-success']")
        avai.append(pr.text)
    except NoSuchElementException:
        avai.append('-')
    
    # scraping if return/exchange is available or not
    try:
        retrn = driver.find_element_by_xpath("//a[@class='a-size-small a-link-normal a-text-normal']")
        ret_exc.append(retrn.text)
    except NoSuchElementException:
        ret_exc.append('-')
        
    # scraping the delivery date of the product    
    try:
        dlvr = driver.find_element_by_xpath("//div[@id='ddmDeliveryMessage']/b")
        exp_del.append(dlvr.text)
    except NoSuchElementException:
        exp_del.append('-')
        
    # scraping the details of the product    
    try:
        dtl = driver.find_element_by_xpath("//ul[@class='a-unordered-list a-vertical a-spacing-mini']/li[2]")
        other_dt.append(dtl.text)
    except NoSuchElementException:
        other_dt.append('---')
    
    # scraping the price of the product
    try:
        p = driver.find_element_by_xpath("//span[@class='a-size-medium a-color-price priceBlockBuyingPriceString']")
        price.append(p.text)
    except NoSuchElementException:
        price.append('-')

In [13]:
amazon = pd.DataFrame({})
amazon['Brand'] = brand_name
amazon['Product'] = product_name
amazon['Rating'] = rating
amazon['Number of ratings'] = num_rating
amazon['Price'] = price
amazon['Return/Exchange'] = ret_exc
amazon['Expected delivery'] = exp_del
amazon['Availability'] = avai
amazon['Other detail'] = other_dt
amazon['Product url'] = product_url

In [14]:
amazon

,Brand,Product,Rating,Number of ratings,Price,Return/Exchange,Expected delivery,Availability,Other detail,Product url
0,NaN,GREY JACK TR90 Material Polarized UV400 Protec...,,3 out of 5,₹ 999.00,10 Days Returns,"Wednesday, June 2",In stock.,Polarized and 100% UV Protection - 100% Protec...,https://www.amazon.in/gp/slredirect/picassoRed...
1,NaN,GREY JACK Sports Polarized Sunglasses for Men ...,,5 out of 5,₹ 999.00,10 Days Returns,"Wednesday, June 2",In stock.,Polarized and 100% UV Protection - 100% Protec...,https://www.amazon.in/gp/slredirect/picassoRed...
2,NaN,GREY JACK TR90 Polarized UV400 Protected Recta...,,5 out of 5,₹ 899.00,10 Days Returns,"Wednesday, June 2",In stock.,Polarized and 100% UV Protection - 100% Protec...,https://www.amazon.in/gp/slredirect/picassoRed...
3,NaN,JIM HALO Retro Square Aviator Sunglasses Premi...,,4.2 out of 5,"₹ 1,299.00",10 Days Returns,"Tuesday, June 1",In stock.,Comfortable Nose Pads Design - adjustable sili...,https://www.amazon.in/JIM-HALO-Aviator-Sunglas...
4,NaN,ELEGANTE Men's Square Sunglasses,,3.5 out of 5,₹ 349.00,10 Days Returns,"Wednesday, June 2",In stock.,Colour : Full Silver Frame & Temple and Single...,https://www.amazon.in/elegante-Branded-Inspire...
...,...,...,...,...,...,...,...,...,...,...
172,NaN,ADDON EYE WEAR Unisex Adult & Unisex Child Squ...,,4 out of 5,₹ 149.00,10 Days Returns,"Friday, June 4",In stock.,Excellent Built Quality For Long Lasting Durab...,https://www.amazon.in/ADDON-EYE-WEAR-Transpare...
173,NaN,Mtv Men - Women Aviator Sunglasses,,3.6 out of 5,"₹ 1,198.00",10 Days Returns,"Tuesday, June 1",In stock.,---,https://www.amazon.in/MTV-Aviator-Sunglass-MTV...
174,NaN,Creek Aviator Sunglasses For Men & Women - Bla...,,-,₹ 399.00,10 Days Returns,"Wednesday, June 2",In stock.,[Super Lightweight] - Super light frame materi...,https://www.amazon.in/Creek-Aviator-Sunglasses...
175,NaN,Carlson Raulen Unisex Adult Rectangular Sungla...,,3.9 out of 5,₹ 349.00,10 Days Returns,"Wednesday, June 2",In stock.,"Light Weight Sunglasses For Men, 100% Uv Prote...",https://aax-eu.amazon.in/x/c/QmxJWhvmbXnj_lov3...


* Q3. Write a python program to access the search bar and search button on images.google.com
and scrape 100 images each for keyword 'fruits', 'cars', and 'Machine Learning'.

In [27]:
# importing library ti sve the image file
import urllib.request

# setting the url for the griver
driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://www.google.co.in/imghp?hl=en&ogbl')

# getting seach bar
search_inp = input('Enter the image name: ')
search_bar = driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
search_bar.send_keys(search_inp)

# getting search button
search_btn = driver.find_element_by_xpath("//span[@class='z1asCe MZy1Rb']")
search_btn.click()

Enter the image name: fruits


In [32]:
# scrolling the page to load more images.

last_height = driver.execute_script('return document.body.scrollHeight')
for i in range(1):
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(2)
    new_height = driver.execute_script('return document.body.scrollHeight')
    if new_height == last_height:
        break
    last_height = new_height

In [39]:
# getting the screenshot of the image and saving it to the local directory.
for i in range(101):
    try:
        driver.find_element_by_xpath('//*[@id="islrg"]/div[1]/div['+str(i)+']/a[1]/div[1]/img').screenshot(r'C:\Users\Aamir Rahman\fruits\f('+str(i)+').png')
    except:
        pass

* Q4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.)
on flipkart.com and scrape the following details for all the search results displayed on 1st page.
Details to be screaped are:-
1. Brand Name.
2. Smartphone name.
4. Colour.
5. RAM.
6. Storage(ROM).
7. Primary camera.
8. Secondary camera.
9. Display size.
10. Display resolution.
11. Processor.
12. Battery Capacity.
13. Price.
14. Product url.
* In case any of the detaail is missing then replace it by '-'. Save your results in a dataframe and csv.

In [4]:
# opening the url
driver.get('https://www.flipkart.com')

# getting the input
search_inp = input("Enter the product: ")

# locating the search bar
search_bar = driver.find_element_by_xpath("//input[@class='_3704LK']")
search_bar.send_keys(search_inp)

# locating the srach button
search_btn = driver.find_element_by_xpath("//button[@class='L0Z3Pu']")
search_btn.click()

Enter the product: pixel 4a


In [29]:
# creating variable tostore our data
brand = []
s_name = []
color = []
ram = []
rom = []
p_camera = []
s_camera = []
display_s = []
display_r = []
soc = []
battery = []
price = []
url = []

In [30]:
# fetching the details

# extracting product name, brand name, colour.
for i in driver.find_elements_by_xpath("//div[@class='_4rR01T']"):
    s_name.append(i.text)
    name = s_name[-1].split() #extracting the brand name from the product name
    brand.append(name[0])
    a = s_name[-1][s_name[-1].find("(")+1:s_name[-1].rfind(")")]
    color.append(a.split(',')[0]) #extracting colour from the product name
    
# extracting the ram and rom.
for j in driver.find_elements_by_xpath("//ul[@class='_1xgFaf']/li[1]"):
    ram.append(j.text.split('|')[0])
    rom.append(j.text.split('|')[1])
    
# extracting primary and secondary camera
for k in driver.find_elements_by_xpath("//ul[@class='_1xgFaf']/li[3]"):
    if(len(k.text.split('|')) == 2):
        p_camera.append(k.text.split('|')[0])
        s_camera.append(k.text.split('|')[1])
    else:
        p_camera.append(k.text.split('|')[0])
        s_camera.append('--')

# extracting display size and display resolution
for l in driver.find_elements_by_xpath("//ul[@class='_1xgFaf']/li[2]"):
    s = l.text[l.text.find("(")+1:l.text.rfind(")")]
    display_s.append(s)
    r = l.text[l.text.find(")")+1:]
    display_r.append(r)
    
# extracting processor name
for m in driver.find_elements_by_xpath("//ul[@class='_1xgFaf']/li[5]"):
    name = m.text
    if 'Processor' in name:
        soc.append(name)
    else:
        soc.append('-')
        
#extracting battery power.
for n in driver.find_elements_by_xpath("//ul[@class='_1xgFaf']/li[4]"):
    battery.append(n.text)
    
# extracting price
for o in driver.find_elements_by_xpath("//div[@class='_30jeq3 _1_WHN1']"):
    price.append(o.text)

# extracting url
for p in driver.find_elements_by_xpath("//a[@class='_1fQZEK']"):
    url.append(p.get_attribute('href'))

In [32]:
flipkart = pd.DataFrame({})
flipkart['Brand'] = brand
flipkart['Product description'] = s_name
flipkart['Colour'] = color
flipkart['RAM'] = ram
flipkart['ROM'] = rom
flipkart['Primary Camera'] = p_camera
flipkart['Secondary Camera'] = s_camera
flipkart['Display Size'] = display_s
flipkart['Display Resolution'] = display_r
flipkart['Processor'] = soc
flipkart['Battery'] = battery
flipkart['Price'] = price
flipkart['Product url'] = url

In [34]:
flipkart.head()

,Brand,Product description,Colour,RAM,ROM,Primary Camera,Secondary Camera,Display Size,Display Resolution,Processor,Battery,Price,Product url
0,Google,"Google Pixel 4a (Just Black, 128 GB)",Just Black,6 GB RAM,128 GB ROM,12.2MP Rear Camera,8MP Front Camera,5.81 inch,Full HD+ Display,Qualcomm Snapdragon 730G Processor,3140 mAh Battery,"₹29,999",https://www.flipkart.com/google-pixel-4a-just-...
1,REDMI,"REDMI Note 9 (Pebble Grey, 64 GB)",Pebble Grey,4 GB RAM,64 GB ROM,48MP + 8MP + 2MP + 2MP,13MP Front Camera,6.53 inch,Full HD+ Display,MediaTek Helio G85 Processor,5020 mAh Battery,"₹10,999",https://www.flipkart.com/redmi-note-9-pebble-g...
2,REDMI,"REDMI 9 Power (Electric Green, 64 GB)",Electric Green,4 GB RAM,64 GB ROM,48MP Rear Camera,8MP Front Camera,6.53 inch,Full HD+ Display,Qualcomm Snapdragon 662 Processor,6000 mAh Battery,"₹10,499",https://www.flipkart.com/redmi-9-power-electri...
3,REDMI,"REDMI 9 Power (Mighty Black, 64 GB)",Mighty Black,4 GB RAM,64 GB ROM,48MP Rear Camera,8MP Front Camera,6.53 inch,Full HD+ Display,Qualcomm Snapdragon 662 Processor,6000 mAh Battery,"₹10,499",https://www.flipkart.com/redmi-9-power-mighty-...
4,REDMI,"REDMI 9 Power (Fiery Red, 64 GB)",Fiery Red,4 GB RAM,64 GB ROM,48MP Rear Camera,8MP Front Camera,6.53 inch,Full HD+ Display,Qualcomm Snapdragon 662 Processor,6000 mAh Battery,"₹10,499",https://www.flipkart.com/redmi-9-power-fiery-r...


* Q5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [26]:
# opening the url

driver.get('https://www.google.com/maps')

search_inp = input("Enter the city: ")
search_bar = driver.find_element_by_xpath("//input[@id='searchboxinput']") # locating search bar
search_bar.send_keys(search_inp)

# locating search button
search_btn = driver.find_element_by_xpath("//button[@id='searchbox-searchbutton']")
search_btn.click()

# getting the geospatial co-ordinates from the url
time.sleep(5)
geo = driver.current_url
r = geo[geo.find("@")+1:]
s = r[0:21]
s = s.split(',')
print('Latitude: {}\nLongitude: {}'.format(s[0], s[1]))

Enter the city: noida
Latitude: 28.5169834
 Longitude: 77.2580416


* Q6.Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [33]:
# opening the url

driver.get('https://trak.in/india-startup-funding-investment-2015/')



In [80]:
sr = []
date = []
startup = []
industry = []
sub_vertical = []
city = []
investor = []
invest_type = []
amount = []

In [83]:
follow_loop = range(1, 11)
for x in follow_loop:
    xpath = "//*[@id='tablepress-48']/tbody/tr["
    xpath += str(x)
    xpath += "]"
    a = driver.find_element_by_xpath(xpath)
    sr.append(a.text)

In [84]:
sr

['1 15/07/2020 Flipkart E-commerce E-commerce Bangalore Walmart Inc\nM&A 1,200,000,000',
 '2 16/07/2020 Vedantu EduTech Online Tutoring Bangalore Coatue Management Series D 100,000,000',
 '3 16/07/2020 Crio EduTech Learning Platform for Developers Bangalore 021 Capital pre-Series A 934,160',
 '4 14/07/2020 goDutch FinTech Group Payments Mumbai Matrix India,Y Combinator, Global Founders Capital, Soma Capital, and VentureSouq. Seed 1,700,000',
 '5 13/07/2020 Mystifly\nAirfare Marketplace Ticketing, Airline Retailing, and Post-Ticketing Services Singapore and Bangalore Recruit Co. Ltd.\n\npre-Series B 3,300,000',
 '6 09/07/2020 JetSynthesys Gaming and Entertainment Gaming and Entertainment\n\nPune Adar Poonawalla and Kris Gopalakrishnan.\nVenture-Series Unknown 400,000',
 '7 10/07/2020 gigIndia\nMarketplace Crowd Sourcing, Freelance Pune Incubate Fund India and Beyond Next Ventures pre-Series A 974,200',
 '8 15/07/2020 PumPumPum Automotive Rental Used Car-leasing platform Gurgaon Early Ad

In [90]:
a = driver.find_element_by_xpath('//*[@id="tablepress-48"]/tbody/tr[3]/td[4]')
a.text

'EduTech'

* Q7. Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [93]:
# passing the url.

driver.get('https://www.digit.in/search/?keyword=top%2010%20gaming%20laptops')

product = []
for i in driver.find_elements_by_xpath('div[@class="searchProduct-desc"]'):
    product.append(i.text)